# Multi-label text classification with HuggingFace Transformers

This notebook demonstrates the use of the HuggingFace
`transformers` library to do perform multi-label text
classification.

## The toxicity dataset

The dataset we'll use is one that Kaggle featured for a
[Toxic Comment Classification Challenge](https://www.kaggle.com/competitions/jigsaw-toxic-comment-classification-challenge/overview). The data are comments from Wikipedia's talk page
edits, where each comment is labeled for different types of
toxicity, including:

* threats
* obscenity
* insults
* identity-based hate

This dataset is a *multi-label* dataset, meaning each comment
can be labeled to contain multiple types of toxicity.

## Libraries used

We'll train our multi-label classification model using HuggingFace
transformers with PyTorch as our deep learning framework.

For preprocessing data we'll use Pandas.

In [1]:
import pandas as pd
import torch

from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification

## Preprocessing the data

In [2]:
df = pd.read_csv('data/train.csv')
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [3]:
df.set_index('id', inplace=True)
df.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
id,,,,,,,
0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [4]:
class CustomDataset(Dataset):
    """Implements a map Dataset for PyTorch."""

    def __init__(
        self,
        dataframe: pd.DataFrame,
        tokenizer: AutoTokenizer,
        max_length: int,
    ):
        """Constructor.

        Args:
            dataframe: The dataframe to source data from.
            tokenizer: The tokenizer to tokenize data.
            max_length: The max length (in tokens) per sample.
        """
        self.tokenizer = tokenizer
        self.dataframe = dataframe
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        comment_text = self.dataframe.comment_text.iloc[index]
        comment_text = ' '.join(comment_text.split())

        inputs = self.tokenizer(
            comment_text,
            None,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
        )
        ids = inputs['input_ids']
        attention_mask = inputs['attention_mask']
        targets = self.dataframe[self.dataframe.columns[1:]].iloc[index]

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'attention_mask': torch.tensor(attention_mask, dtype=torch.long),
            'targets': torch.tensor(targets, dtype=torch.float)
        }

In [18]:
tokenizer = AutoTokenizer.from_pretrained(
    'distilbert-base-uncased',
    use_fast=True,
)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [6]:
train_dataset = df.sample(frac=0.8)

test_dataset = df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

training_set = CustomDataset(train_dataset, tokenizer, 512)
testing_set = CustomDataset(test_dataset, tokenizer, 512)

In [7]:
training_loader = DataLoader(
    training_set,
    batch_size=8,
    shuffle=True,
    num_workers=0,
)
testing_loader = DataLoader(
    training_set,
    batch_size=4,
    shuffle=True,
    num_workers=0,
)

## Creating the classification model

In [19]:
model = AutoModelForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=6,
)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias', 'pre_classifier

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [10]:
def loss_fn(outputs, targets):
    """The loss function.

    Args:
        outputs: Model outputs.
        targets: Model targets.
    """
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

In [11]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-5)

## Training the model

In [12]:
def train(epoch: int):
    """Training function per epoch.

    Args:
        epoch: Epoch number.
    """

    model.train()
    for data in training_loader:
        ids = data['ids'].to(device, dtype=torch.long)
        attention_mask = data['attention_mask'].to(device, dtype=torch.long)
        targets = data['targets'].to(device, dtype=torch.float)

        outputs = model(ids, attention_mask)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f'Epoch: {epoch}, Loss: {loss.item()}')

In [13]:
for epoch in range(1, 3):
    train(epoch)

TypeError: dropout(): argument 'input' (position 1) must be Tensor, not SequenceClassifierOutput